In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [32]:
#Usaremos el CSV "AllDecks" tras la modificaciones realizadas previamente
df = pd.read_csv('../CSV/AllDecks.csv')
df.head()

,code,mainBoard,meta,name,releaseDate,sideBoard,type,total_price,deck_color
0,KTK,"[{'artist': 'Zack Stella', 'borderColor': 'bla...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Abzan Siege,2014-09-26,[],Intro Pack,0.68,"['B', 'W', 'G']"
1,C18,"[{'artist': 'Winona Nelson', 'borderColor': 'b...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Adaptive Enchantment,2018-08-10,"[{'artist': 'Johannes Voss', 'borderColor': 'b...",Commander Deck,0.00,"['W', 'U', 'G']"
2,BTD,"[{'artist': 'Doug Chaffee', 'borderColor': 'wh...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Aerodoom,2000-10-01,[],Theme Deck,0.00,"['B', 'U']"
3,WTH,"[{'artist': 'Douglas Shuler', 'borderColor': '...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Air Forces,2007-12-12,[],MTGO Theme Deck,2.63,"['U', 'W']"
4,JUD,"[{'artist': 'Ron Spears', 'borderColor': 'blac...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Air Razers,2002-05-27,[],Theme Deck,0.68,"['R', 'U']"


In [36]:
valores_colores = {
    'B': 5,
    'W': 3,
    'G': 4,
    'R': 2,
    'U': 1
}

In [48]:
def calc_cmb_colors(colors):
    return sum(valores_colores.get(color, 0) for color in colors)

df['color_score'] = df['deck_color'].apply(calc_cmb_colors)

In [50]:
df.head()

,code,mainBoard,meta,name,releaseDate,sideBoard,type,total_price,deck_color,color_score
0,KTK,"[{'artist': 'Zack Stella', 'borderColor': 'bla...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Abzan Siege,2014-09-26,[],Intro Pack,0.68,"['B', 'W', 'G']",12
1,C18,"[{'artist': 'Winona Nelson', 'borderColor': 'b...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Adaptive Enchantment,2018-08-10,"[{'artist': 'Johannes Voss', 'borderColor': 'b...",Commander Deck,0.00,"['W', 'U', 'G']",8
2,BTD,"[{'artist': 'Doug Chaffee', 'borderColor': 'wh...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Aerodoom,2000-10-01,[],Theme Deck,0.00,"['B', 'U']",6
3,WTH,"[{'artist': 'Douglas Shuler', 'borderColor': '...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Air Forces,2007-12-12,[],MTGO Theme Deck,2.63,"['U', 'W']",4
4,JUD,"[{'artist': 'Ron Spears', 'borderColor': 'blac...","{'date': '2019-11-10', 'pricesDate': '2019-11-...",Air Razers,2002-05-27,[],Theme Deck,0.68,"['R', 'U']",3


In [68]:
X = df[['name','color_score']]
y = df['total_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Liliana'